In [ ]:
# | default_exp data


In [ ]:
# |hide

from nbdev.showdoc import *


In [ ]:
# |export

import os

import pandas as pd

_THIS_DIR = os.path.abspath(os.path.dirname(os.path.abspath("")))


# Data

> Helpers for loading data


## Polymers


In [ ]:
# |export
def get_polymer_data(
    datadir="../data" # path to folder with data files
):
    return pd.read_csv(os.path.join(datadir, "polymers.csv"))


In [ ]:
df = get_polymer_data("../data/")


In [ ]:
df.head()


,Unnamed: 0,smiles,string,deltaGmin,A2_normalized,deltaGmin_cat,A2_normalized_cat,num_[W],max_[W],num_[Tr],...,[W],[W].1,[Tr],[Tr].1,[Ta],[Ta].1,[R],[R].1,rel_shannon,length
0,0,[W][Ta][Tr][W][W][Ta][Ta][Ta][R][W][Tr][Tr][R]...,W-A-B-W-W-A-A-A-R-W-B-B-R-R-B-R,-7.535286,-0.109726,very large,very small,0.25,2,0.250000,...,4.0,0.25,4.0,0.25,4.0,0.25,4.0,0.25,0.5,16
1,1,[R][W][W][R][R][Tr][Tr][Tr][Ta][Ta][Ta][W][W][...,R-W-W-R-R-B-B-B-A-A-A-W-W-A-R-B,-7.270527,0.580595,very large,very large,0.40,2,0.200000,...,4.0,0.25,4.0,0.25,4.0,0.25,4.0,0.25,0.5,16
2,2,[Ta][R][Ta][W][Tr][W][Ta][R][Tr][W][Ta][Tr][Tr...,A-R-A-W-B-W-A-R-B-W-A-B-B-R-W-R,-6.416311,0.956320,very large,very large,0.00,0,1.000000,...,4.0,0.25,4.0,0.25,4.0,0.25,4.0,0.25,0.5,16
3,3,[W][Ta][R][Ta][Tr][Tr][Tr][W][Ta][W][Tr][R][Ta...,W-A-R-A-B-B-B-W-A-W-B-R-A-W-R-R,-6.684816,1.129924,very large,very large,0.00,0,0.500000,...,4.0,0.25,4.0,0.25,4.0,0.25,4.0,0.25,0.5,16
4,4,[R][R][Tr][Tr][W][R][Ta][W][R][W][Ta][Tr][Ta][...,R-R-B-B-W-R-A-W-R-W-A-B-A-A-W-B,-6.606492,-0.496439,very large,very small,0.00,0,0.333333,...,4.0,0.25,4.0,0.25,4.0,0.25,4.0,0.25,0.5,16


## Photoswitch


In [ ]:
# |export
def get_photoswitch_data(
    datadir="../data" # path to folder with data files
):
    return pd.read_csv(os.path.join(datadir, "photoswitches.csv"))


In [ ]:
df = get_photoswitch_data("../data/")


In [ ]:
df.head()


,SMILES,rate of thermal isomerisation from Z-E in s-1,Solvent used for thermal isomerisation rates,Z PhotoStationaryState,E PhotoStationaryState,E isomer pi-pi* wavelength in nm,Extinction,E isomer n-pi* wavelength in nm,Extinction coefficient in M-1 cm-1,Z isomer pi-pi* wavelength in nm,...,CAM-B3LYP/6-31G** DFT E isomer n-pi* wavelength in nm,CAM-B3LYP/6-31G** DFT Z isomer pi-pi* wavelength in nm,CAM-B3LYP/6-31G** DFT Z isomer n-pi* wavelength in nm,BHLYP/6-31G* DFT E isomer pi-pi* wavelength in nm,BHLYP/6-31G* DFT E isomer n-pi* wavelength in nm,BHLYP/6-31G* Z isomer pi-pi* wavelength in nm,BHLYP/6-31G* DFT Z isomer n-pi* wavelength in nm,name,selfies,wavelength_cat
0,C[N]1N=NC(=N1)N=NC2=CC=CC=C2,2.100000e-07,MeCN,76.0,72.0,310.0,1.67,442.0,0.0373,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[C][NH0][N][=N][C][=Branch1][Ring2][=N][Ring1]...,0.0
1,C[N]1C=NC(=N1)N=NC2=CC=CC=C2,3.800000e-07,MeCN,90.0,84.0,310.0,1.87,438.0,0.0505,272.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[C][NH0][C][=N][C][=Branch1][Ring2][=N][Ring1]...,0.0
2,C[N]1C=CC(=N1)N=NC2=CC=CC=C2,1.100000e-07,MeCN,98.0,97.0,320.0,1.46,425.0,0.0778,272.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[C][NH0][C][=C][C][=Branch1][Ring2][=N][Ring1]...,0.0
3,C[N]1C=C(C)C(=N1)N=NC2=CC=CC=C2,1.500000e-06,MeCN,96.0,87.0,325.0,1.74,428.0,0.0612,286.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[C][NH0][C][=C][Branch1][C][C][C][=Branch1][Ri...,0.0
4,C[N]1C=C(C=N1)N=NC2=CC=CC=C2,7.600000e-09,MeCN,98.0,70.0,328.0,1.66,417.0,0.0640,275.0,...,427.0,256.0,401.0,NaN,NaN,NaN,NaN,NaN,[C][NH0][C][=C][Branch1][Branch1][C][=N][Ring1...,0.0
